# Logging results and uploading models to Weights & Biases
In this example, we train a simple XGBoost model and log the training
results to Weights & Biases. We also save the resulting model checkpoints
as artifacts.

There are two ways to achieve this:

1. Automatically using the `ray.air.callbacks.wandb.WandbLoggerCallback`
2. Manually using the `wandb` API

This tutorial will walk you through both options.

Let's start with installing our dependencies:

In [1]:
!pip install -qU "ray[tune]" sklearn xgboost_ray wandb

Then we need some imports:

In [3]:
import ray

from ray.air.config import RunConfig, ScalingConfig
from ray.air.result import Result
from ray.air.callbacks.wandb import WandbLoggerCallback

We define a simple function that returns our training dataset as a Ray Dataset:


In [4]:
def get_train_dataset() -> ray.data.Dataset:
    dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")
    return dataset

And that's the common parts. We now dive into the two options to interact with Weights and Biases.

## Using the WandbLoggerCallback

The WandbLoggerCallback does all the logging and reporting for you. It is especially useful when you use an out-of-the-box trainer like `XGBoostTrainer`. In these trainers, you don't define your own training loop, so using the AIR W&B callback is the best way to log your results to Weights and Biases.

First we define a simple training function.

All the magic happens within the `WandbLoggerCallback`:

```python
WandbLoggerCallback(
    project=wandb_project,
    save_checkpoints=True,
)
```

It will automatically log all results to Weights & Biases and upload the checkpoints as artifacts. It assumes you're logged in into Wandb via an API key or `wandb login`.

In [3]:
from ray.train.xgboost import XGBoostTrainer


def train_model_torch(train_dataset: ray.data.Dataset, wandb_project: str) -> Result:
    """Train a simple XGBoost model and return the result."""
    trainer = XGBoostTrainer(
        scaling_config=ScalingConfig(num_workers=2),
        params={"tree_method": "auto"},
        label_column="target",
        datasets={"train": train_dataset},
        num_boost_round=10,
        run_config=RunConfig(
            callbacks=[
                # This is the part needed to enable logging to Weights & Biases.
                # It assumes you've logged in before, e.g. with `wandb login`.
                WandbLoggerCallback(
                    project=wandb_project,
                    save_checkpoints=True,
                )
            ]
        ),
    )
    result = trainer.fit()
    return result

Let's kick off a run:

In [5]:
wandb_project = "ray_air_example"

train_dataset = get_train_dataset()
result = train_model_xgboost(train_dataset=train_dataset, wandb_project=wandb_project)

2022-05-19 15:22:11,956	INFO services.py:1483 -- View the Ray dashboard at http://127.0.0.1:8266
2022-05-19 15:22:15,995	INFO wandb.py:172 -- Already logged into W&B.


Trial name,status,loc,iter,total time (s),train-rmse
XGBoostTrainer_14a73_00000,TERMINATED,127.0.0.1:20065,10,10.2724,0.030717


(raylet) 2022-05-19 15:22:17,422	INFO context.py:70 -- Exec'ing worker with command: exec /Users/kai/.pyenv/versions/3.7.7/bin/python3.7 /Users/kai/coding/ray/python/ray/workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=61838 --object-store-name=/tmp/ray/session_2022-05-19_15-22-09_017478_19912/sockets/plasma_store --raylet-name=/tmp/ray/session_2022-05-19_15-22-09_017478_19912/sockets/raylet --redis-address=None --storage=None --temp-dir=/tmp/ray --metrics-agent-port=63609 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:62933 --redis-password=5241590000000000 --startup-token=16 --runtime-env-hash=-2010331134
wandb: Currently logged in as: kaifricke. Use `wandb login --relogin` to force relogin
(GBDTTrainable pid=20065) UserWarning: Dataset 'train' has 1 blocks, which is less than the `num_workers` 2. This dataset will be automatically repartitioned to 2 blocks.
(raylet) 2022-05-19 15:22:23,215	INFO context.py:70 -- Exec'

(GBDTTrainable pid=20065) 2022-05-19 15:22:24,711	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(raylet) 2022-05-19 15:22:26,090	INFO context.py:70 -- Exec'ing worker with command: exec /Users/kai/.pyenv/versions/3.7.7/bin/python3.7 /Users/kai/coding/ray/python/ray/workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=61838 --object-store-name=/tmp/ray/session_2022-05-19_15-22-09_017478_19912/sockets/plasma_store --raylet-name=/tmp/ray/session_2022-05-19_15-22-09_017478_19912/sockets/raylet --redis-address=None --storage=None --temp-dir=/tmp/ray --metrics-agent-port=63609 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:62933 --redis-password=5241590000000000 --startup-token=18 --runtime-env-hash=-2010331069
(raylet) 2022-05-19 15:22:26,234	INFO context.py:70 -- Exec'ing worker with command: exec /Users/kai/.pyenv/versions/3.7.7/bin/python3.7 /Users/kai/cod

Result for XGBoostTrainer_14a73_00000:
  date: 2022-05-19_15-22-31
  done: false
  experiment_id: 2d50bfe80d2a441e80f4ca05f7c3b607
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 20065
  should_checkpoint: true
  time_since_restore: 10.080440044403076
  time_this_iter_s: 10.080440044403076
  time_total_s: 10.080440044403076
  timestamp: 1652970151
  timesteps_since_restore: 0
  train-rmse: 0.357284
  training_iteration: 1
  trial_id: 14a73_00000
  warmup_time: 0.006903171539306641
  


wandb: Adding directory to artifact (/Users/kai/ray_results/XGBoostTrainer_2022-05-19_15-22-14/XGBoostTrainer_14a73_00000_0_2022-05-19_15-22-16/checkpoint_000001)... Done. 0.1s
(GBDTTrainable pid=20065) 2022-05-19 15:22:32,051	INFO main.py:1519 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 7.37 seconds (2.79 pure XGBoost training time).
wandb: Adding directory to artifact (/Users/kai/ray_results/XGBoostTrainer_2022-05-19_15-22-14/XGBoostTrainer_14a73_00000_0_2022-05-19_15-22-16/checkpoint_000002)... Done. 0.1s
wandb: Adding directory to artifact (/Users/kai/ray_results/XGBoostTrainer_2022-05-19_15-22-14/XGBoostTrainer_14a73_00000_0_2022-05-19_15-22-16/checkpoint_000003)... Done. 0.1s
wandb: Adding directory to artifact (/Users/kai/ray_results/XGBoostTrainer_2022-05-19_15-22-14/XGBoostTrainer_14a73_00000_0_2022-05-19_15-22-16/checkpoint_000004)... Done. 0.1s
wandb: Adding directory to artifact (/Users/kai/ray_results/XGBoostTrainer_2022-05-19_15-22-14/XG

Result for XGBoostTrainer_14a73_00000:
  date: 2022-05-19_15-22-32
  done: true
  experiment_id: 2d50bfe80d2a441e80f4ca05f7c3b607
  experiment_tag: '0'
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 20065
  should_checkpoint: true
  time_since_restore: 10.272444248199463
  time_this_iter_s: 0.023891210556030273
  time_total_s: 10.272444248199463
  timestamp: 1652970152
  timesteps_since_restore: 0
  train-rmse: 0.030717
  training_iteration: 10
  trial_id: 14a73_00000
  warmup_time: 0.006903171539306641
  


2022-05-19 15:22:42,727	INFO tune.py:753 -- Total run time: 27.83 seconds (26.61 seconds for the tuning loop).


iterations_since_restore,▁▂▃▃▄▅▆▆▇█
time_since_restore,▁▂▃▃▄▅▅▆▇█
time_this_iter_s,█▁▁▁▁▁▁▁▁▁
time_total_s,▁▂▃▃▄▅▅▆▇█
timestamp,▁▁▁▁▁▁▁▁██
timesteps_since_restore,▁▁▁▁▁▁▁▁▁▁
train-rmse,█▆▄▃▂▂▂▁▁▁
training_iteration,▁▂▃▃▄▅▆▆▇█
warmup_time,▁▁▁▁▁▁▁▁▁▁
iterations_since_restore,10
time_since_restore,10.27244


Check out your [WandB](https://wandb.ai/) project to see the results!

## Using the `wandb` API

When you define your own training loop, you sometimes want to manually interact with the Weights and Biases API. Ray AIR provides a `setup_wandb()` function that takes care of the initialization.

The main benefit here is that authentication to Weights and Biases is automatically set up for you, and sensible default names for your runs are set. Of course, you can override these.

Additionally in distributed training you often only want to report the results of the rank 0 worker. This can also be done automatically using our setup.

Let's define a distributed training loop:

In [20]:
from ray.air import session
from ray.air.integrations.wandb import setup_wandb

import torch.optim as optim
import torch.nn as nn

def train_loop(config):
    wandb = setup_wandb(config)
    
    dataset = session.get_dataset_shard("train")

    model = nn.Linear(10, 2)

    optimizer = optim.SGD(
        model.parameters(),
        lr=config.get("lr", 0.01),
    )
    loss = nn.CrossEntropyLoss()
    
    for batch in dataset.iter_torch_batches():
        y = batch["target"]
        X = batch[batch.columns.difference(["target"])]
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        session.report({"loss": loss.item()})
    

In [21]:
from ray.train.torch import TorchTrainer


def train_model_torch(train_dataset: ray.data.Dataset, wandb_project: str) -> Result:
    """Train a simple XGBoost model and return the result."""
    trainer = TorchTrainer(
        train_loop_per_worker=train_loop,
        scaling_config=ScalingConfig(num_workers=2),
        train_loop_config={"lr": 0.01, "wandb": {"project": wandb_project}},
        datasets={"train": train_dataset},
    )
    result = trainer.fit()
    return result

In [22]:
wandb_project = "ray_air_example_torch"

train_dataset = get_train_dataset()
result = train_model_torch(train_dataset=train_dataset, wandb_project=wandb_project)

2022-10-28 15:21:28,477	WARNING read_api.py:297 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.


(RayTrainWorker pid=87002) 2022-10-28 15:21:36,578	INFO config.py:88 -- Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=87002) 2022-10-28 15:21:38,604	INFO wandb.py:214 -- Already logged into W&B.
(RayTrainWorker pid=87003) /Users/kai/coding/ray/python/ray/air/_internal/torch_utils.py:135: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:178.)
(RayTrainWorker pid=87003)   return torch.as_tensor(ndarray, dtype=dtype, device=device)
(RayTrainWorker pid=87002) wandb: Currently logged in as: kaifricke (use `wandb login --relogin` to force relogin)


(RayTrainWorker pid=87002) signal only works in main thread


(RayTrainWorker pid=87002) wandb: wandb version 0.13.4 is available!  To upgrade, please run:
(RayTrainWorker pid=87002) wandb:  $ pip install wandb --upgrade
(RayTrainWorker pid=87002) wandb: Tracking run with wandb version 0.12.15
(RayTrainWorker pid=87002) wandb: Run data is saved locally in /Users/kai/ray_results/TorchTrainer_2022-10-28_15-21-28/TorchTrainer_de413_00000_0_2022-10-28_15-21-28/rank_0/wandb/run-20221028_152138-de413_00000
(RayTrainWorker pid=87002) wandb: Run `wandb offline` to turn off syncing.
(RayTrainWorker pid=87002) wandb: Syncing run TorchTrainer_de413_00000
(RayTrainWorker pid=87002) wandb: ⭐️ View project at https://wandb.ai/kaifricke/ray_air_example_torch
(RayTrainWorker pid=87002) wandb: 🚀 View run at https://wandb.ai/kaifricke/ray_air_example_torch/runs/de413_00000
(RayTrainWorker pid=87002) /Users/kai/coding/ray/python/ray/air/_internal/torch_utils.py:135: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensor

Trial name,date,experiment_id,hostname,node_ip,pid,timestamp,trial_id
TorchTrainer_de413_00000,2022-10-28_15-21-33,656a990960d84bf4b6f7721171ee6681,Kais-MBP.local.meter,127.0.0.1,86986,1666995693,de413_00000


2022-10-28 15:21:44,924	ERROR tune.py:783 -- Trials did not complete: [TorchTrainer_de413_00000]
2022-10-28 15:21:44,924	INFO tune.py:788 -- Total run time: 16.09 seconds (15.97 seconds for the tuning loop).


RayTaskError(AttributeError): [36mray::_Inner.train()[39m (pid=86986, ip=127.0.0.1, repr=TorchTrainer)
  File "/Users/kai/coding/ray/python/ray/tune/trainable/trainable.py", line 361, in train
    raise skipped from exception_cause(skipped)
  File "/Users/kai/coding/ray/python/ray/train/_internal/utils.py", line 54, in check_for_failure
    ray.get(object_ref)
ray.exceptions.RayTaskError(AttributeError): [36mray::RayTrainWorker._RayTrainWorker__execute()[39m (pid=87003, ip=127.0.0.1, repr=<ray.train._internal.worker_group.RayTrainWorker object at 0x114d307d0>)
  File "/Users/kai/coding/ray/python/ray/train/_internal/worker_group.py", line 31, in __execute
    raise skipped from exception_cause(skipped)
  File "/Users/kai/coding/ray/python/ray/train/_internal/utils.py", line 129, in discard_return_wrapper
    train_func(*args, **kwargs)
  File "/var/folders/b2/0_91bd757rz02lrmr920v0gw0000gn/T/ipykernel_79754/1048726765.py", line 22, in train_loop
AttributeError: 'dict' object has no